# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [1]:
!nvidia-smi

Tue Mar 21 06:47:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.29                 Driver Version: 531.29       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 L...  WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   34C    P8               20W /  N/A|    553MiB / 16384MiB |     25%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the links are dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace, or you can use [the Kaggle API](https://www.kaggle.com/general/74235) to directly download the data into colab.**


### Download train/test metadata

In [2]:
# Main link
# !wget -O libriphone.zip "https://github.com/xraychen/shiny-robot/releases/download/v1.0/libriphone.zip"

# Backup Link 0
# !pip install --upgrade gdown
# !gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
# !pip install --upgrade gdown
# !gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Backup link 2
# !wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

# Backup link 3
# !wget -O libriphone.zip "https://www.dropbox.com/s/p2ljbtb2bam13in/libriphone.zip?dl=1"

# !unzip -q libriphone.zip
# !ls libriphone

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [3]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

# def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337, for_rnn=False):
#     class_num = 41 # NOTE: pre-computed, should not need change
#     mode = 'train' if (split == 'train' or split == 'val') else 'test'

#     label_dict = {}
#     if mode != 'test':
#       phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

#       for line in phone_file:
#           line = line.strip('\n').split(' ')
#           label_dict[line[0]] = [int(p) for p in line[1:]]

#     if split == 'train' or split == 'val':
#         # split training and validation data
#         usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
#         random.seed(train_val_seed)
#         random.shuffle(usage_list)
#         percent = int(len(usage_list) * train_ratio)
#         usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
#     elif split == 'test':
#         usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
#     else:
#         raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

#     usage_list = [line.strip('\n') for line in usage_list]
#     print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

#     max_len = 3000000
#     X = torch.empty(max_len, 39 * concat_nframes)
#     if mode != 'test':
#         y = torch.empty(max_len, dtype=torch.long) if not for_rnn else torch.empty(max_len, concat_nframes, dtype=torch.long)

#     idx = 0
#     for i, fname in tqdm(enumerate(usage_list)):
#         feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
#         cur_len = len(feat)
#         feat = concat_feat(feat, concat_nframes)
#         if mode != 'test':
#           label = torch.LongTensor(label_dict[fname])
#           if for_rnn:
#               label = torch.unsqueeze(label, 1)
#               label = concat_feat(label, concat_nframes)

#         X[idx: idx + cur_len, :] = feat
#         if mode != 'test':
#           y[idx: idx + cur_len] = label

#         idx += cur_len

#     X = X[:idx, :]
#     if mode != 'test':
#       y = y[:idx]
    
#     if for_rnn:
#         X = X.view(idx, concat_nframes, -1)

#     print(f'[INFO] {split} set')
#     print(X.shape)
#     if mode != 'test':
#       print(y.shape)
#       return X, y
#     else:
#       return X

def preprocess_data(split, feat_dir, phone_path, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))
    
    X = []
    if mode != 'test':
        y = []
    
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])
            
        X.append(feat)
        if mode != 'test':
            y.append(label)
    
    if mode != 'test':
        return X, y
    else:
        return X
        
    # max_len = 3000000
    # X = torch.empty(max_len, 39 * concat_nframes)
    # if mode != 'test':
    #     y = torch.empty(max_len, dtype=torch.long) if not for_rnn else torch.empty(max_len, concat_nframes, dtype=torch.long)

    # idx = 0
    # for i, fname in tqdm(enumerate(usage_list)):
    #     feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
    #     cur_len = len(feat)
    #     feat = concat_feat(feat, concat_nframes)
    #     if mode != 'test':
    #       label = torch.LongTensor(label_dict[fname])
    #       if for_rnn:
    #           label = torch.unsqueeze(label, 1)
    #           label = concat_feat(label, concat_nframes)

    #     X[idx: idx + cur_len, :] = feat
    #     if mode != 'test':
    #       y[idx: idx + cur_len] = label

    #     idx += cur_len

    # X = X[:idx, :]
    # if mode != 'test':
    #   y = y[:idx]
    
    # if for_rnn:
    #     X = X.view(idx, concat_nframes, -1)

    # print(f'[INFO] {split} set')
    # print(X.shape)
    # if mode != 'test':
    #   print(y.shape)
    #   return X, y
    # else:
    #   return X

def train_pad_collate_fn(batch):
    
    Xs, ys = zip(*batch)
    masks = [ torch.ones_like(y) for y in ys ]
    Xs = pad_sequence(Xs, batch_first=True)
    ys = pad_sequence(ys, batch_first=True)
    masks = pad_sequence(masks, batch_first=True)
    
    return Xs, ys, masks

def test_pad_collate_fn(batch):
    
    Xs = batch
    masks = [ torch.ones((X.shape[0],)) for X in Xs ]
    Xs = pad_sequence(Xs, batch_first=True)
    masks = pad_sequence(masks, batch_first=True)
    
    return Xs, masks


## Define Dataset

In [4]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            # self.label = torch.LongTensor(y)
            self.label = y
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


## Define Model

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from bi_lstm_crf import CRF

# class BasicBlock(nn.Module):
#     def __init__(self, input_dim, output_dim, drop_rate=0.1):
#         super(BasicBlock, self).__init__()

#         self.block = nn.Sequential(
#             nn.Linear(input_dim, output_dim),
#             nn.BatchNorm1d(output_dim),
#             nn.Dropout(drop_rate),
#             nn.ReLU(),
#         )

#     def forward(self, x):
#         x = self.block(x)
#         return x

# class Classifier(nn.Module):
#     def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256, drop_rate=0.1):
#         super(Classifier, self).__init__()

#         self.fc = nn.Sequential(
#             BasicBlock(input_dim, hidden_dim, drop_rate=drop_rate),
#             *[BasicBlock(hidden_dim, hidden_dim, drop_rate=drop_rate) for _ in range(hidden_layers)],
#             nn.Linear(hidden_dim, output_dim)
#         )

#     def forward(self, x):
#         x = self.fc(x)
#         return x

class RNNClassifier(nn.Module):
    def __init__(self, input_dim, out_dim=41, hidden_layers=1, hidden_dim=256, drop_rate=0.1):
        super(RNNClassifier, self).__init__()
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, hidden_layers, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.af = nn.Sequential(
            nn.LazyBatchNorm1d(),
            # nn.LayerNorm(2*hidden_dim),
            # nn.GELU()
        )
        self.fc = nn.LazyLinear(out_dim)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.af(x)
        x = self.fc(x)
        return x
    
    
class BiLSTMCRF(nn.Module):
    def __init__(self, input_dim, out_dim=41, hidden_layers=1, hidden_dim=256, drop_rate=0.1, device="cpu"):
        super(BiLSTMCRF, self).__init__()
        self.bilstm = nn.LSTM(input_dim, hidden_dim, hidden_layers, dropout=drop_rate, bidirectional=True, batch_first=True)
        self.crf = CRF(hidden_dim * 2, out_dim)
        self.device = device
        self.to(device)
        
    def loss(self, x, y, masks):
        x, y = x.to(self.device), y.to(self.device)
        features, _ = self.bilstm(x)
        if masks is None:
            masks = torch.ones_like(y, device=self.device)
        loss = self.crf.loss(features, y, masks)
        return loss
    
    def forward(self, x, masks=None):
        x = x.to(self.device)
        features, _ = self.bilstm(x)
        if masks is None:
            masks = torch.ones(x.shape[:2], device=self.device)
        scores, outputs = self.crf(features, masks)
        return scores, outputs
    
    
# class BiLSTMCRF(nn.Module):
#     def __init__(self, input_dim, out_dim=41, hidden_layers=1, hidden_dim=256, drop_rate=0.1, device="cpu"):
#         super(BiLSTMCRF, self).__init__()
        
#         self._BIG_NEG = -1e5
#         self.out_dim = out_dim
#         self.bilstm = nn.LSTM(input_dim, hidden_dim, hidden_layers, dropout=drop_rate, bidirectional=True, batch_first=True)
#         self.fc = nn.LazyLinear(out_dim)
#         self.trans = nn.Parameter(torch.randn(out_dim+1, out_dim+1))
#         self.trans.data[out_dim, :] = self._BIG_NEG
#         self.device = device
#         self.to(device)
#         # self.trans.data[:,out_dim+1] = -1e5
    
#     def _get_bilstm_freatures(self, input):
#         bilstm_out, _ = self.bilstm(input)
#         bilstm_feats = self.fc(bilstm_out)
#         return bilstm_feats # (batch_size, concat_nframes, out_dim)
    
#     def _score(self, feats, labels):
#         batch_size = feats.shape[0]
#         score = torch.zeros(batch_size, 1).to(self.device)
#         labels = torch.cat([torch.tensor([[self.out_dim]] * batch_size).to(self.device), labels], dim=-1)
#         for i in range(feats.shape[-2]):
#             score += (self.trans[ labels[:, i+1], labels[:, i]] + feats[range(batch_size), i, labels[:, i+1]]).view(-1,1)
#         # score += self.trans[ -1,  labels[:, -1] ]
#         return score # (batch_size, 1)
        
#     def _log_sum_exp_score(self, feats):
#         batch_size = feats.shape[0]
#         forward_var = torch.full((batch_size, self.out_dim+1), self._BIG_NEG).to(self.device)
#         forward_var[:, self.out_dim] = 0.
        
#         for i in range(feats.shape[-2]):
#             alpha_t = []
#             for next_label in range(self.out_dim+1):
#                 emit_score = self._BIG_NEG if next_label == self.out_dim else feats[:, i, next_label].view(-1, 1)
#                 trans_score = self.trans[next_label]
#                 next_label_var = forward_var + (emit_score + trans_score)
#                 alpha_t.append(torch.logsumexp(next_label_var, dim=-1, keepdim=True))
#             forward_var = torch.cat(alpha_t, dim=-1)
#         alpha = torch.logsumexp(forward_var, dim=-1, keepdim=True)
#         return alpha # (batch_size, 1)
    
#     def _viterbi_decode(self, feats):
#         backpointers = []
#         batch_size = feats.shape[0]
#         forward_var = torch.full((batch_size, self.out_dim+1), self._BIG_NEG).to(self.device)
#         forward_var[:, self.out_dim] = 0.
        
#         for i in range(feats.shape[-2]):
#             bptrs_t = []
#             viterbivars_t = []
#             for next_label in range(self.out_dim+1):
#                 next_label_var = forward_var + self.trans[next_label]
#                 best_label = torch.argmax(next_label_var, dim=-1, keepdim=True)
#                 bptrs_t.append(best_label)
#                 viterbivars_t.append(next_label_var[range(batch_size), best_label.squeeze()].view(-1,1))
#             feat = feats[:, i, :] 
#             feat = torch.cat([feat, torch.full((batch_size, 1), self._BIG_NEG).to(self.device)], dim=-1)
#             forward_var = torch.cat(viterbivars_t, dim=-1) + feat
#             backpointers.append(torch.cat(bptrs_t, dim=-1))
            
#         best_label = torch.argmax(forward_var, dim=-1, keepdim=True)
#         path_score = forward_var[range(batch_size), best_label.squeeze()].view(-1,1)
#         best_path = [ best_label ]
        
#         for bptrs_t in reversed(backpointers):
#             best_label = bptrs_t[range(batch_size), best_label.squeeze()].view(-1,1)
#             best_path.append(best_label)
            
#         best_path.reverse()
#         best_path = torch.cat(best_path, dim=-1)
#         return path_score, best_path[:,1:]
                
#     def neg_log_likelihood(self, input, labels):
#         feats = self._get_bilstm_freatures(input)
#         forward_score = self._log_sum_exp_score(feats)
#         gt_score = self._score(feats, labels)
#         return torch.mean(forward_score - gt_score)
                 
#     def forward(self, x):
#         x = self._get_bilstm_freatures(x)
#         _, x = self._viterbi_decode(x)
#         return x



## Hyper-parameters

In [6]:
# data prarameters
concat_nframes = 1 # 25 # 1     # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.9 # 0.8         # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 0                        # random seed
batch_size = 48 # 1024          # batch size
num_epoch = 800                 # the number of training epoch
learning_rate = 1e-3            # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved
drop_rate = 0.5                 # the probablity of drop out (Customized)
early_stop = 8                  # If model has not improved for this many consecutive epochs, stop training. (Customized)
weight_decay = 5e-2             # weight decay for optimizer (Customized)

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 6 # 1           # the number of hidden layers
hidden_dim = 256                # the hidden dim

## Prepare dataset and model

In [7]:
import gc

# preprocess data
# train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, for_rnn=True)
# val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio, for_rnn=True)
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=train_pad_collate_fn)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, collate_fn=train_pad_collate_fn)

[Dataset] - # phone classes: 41, number of utterances for train: 3857


3857it [00:01, 3133.47it/s]


[Dataset] - # phone classes: 41, number of utterances for val: 429


429it [00:00, 2876.71it/s]


In [8]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda:0


In [9]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [10]:

# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim, drop_rate=drop_rate).to(device)
# model = RNNClassifier(input_dim=input_dim//concat_nframes, hidden_layers=hidden_layers, hidden_dim=hidden_dim, drop_rate=drop_rate).to(device)
model = BiLSTMCRF(input_dim=input_dim//concat_nframes, hidden_layers=hidden_layers, hidden_dim=hidden_dim, drop_rate=drop_rate, device=device)
# criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=2, T_mult=2) # Customized learning rate scheduler

## Training

In [11]:
best_acc = 0.0
early_stop_count = 0
for epoch in range(num_epoch):
    train_acc = 0.0
    # train_mid_acc = 0.0
    train_all = 0.0
    train_loss = 0.0
    val_acc = 0.0
    # val_mid_acc = 0.0
    val_all = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels, masks = batch
        features = features.to(device)
        labels = labels.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad() 
        # outputs = model(features) 
        _, outputs = model(features, masks)
        outputs = [ torch.Tensor(x) for x in outputs ]
        outputs = pad_sequence(outputs, batch_first=True)
        
        # loss = criterion(outputs, labels) 
        # loss = criterion(outputs.transpose(1,-1), labels)
        # loss = model.neg_log_likelihood(features, labels)
        loss = model.loss(features, labels, masks)
        loss.backward() 
        optimizer.step() 
        
        # _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        # _, train_pred = torch.max(outputs, -1)
        train_pred = torch.Tensor(outputs).to(device)
        train_acc += ((train_pred.detach() == labels.detach()) * masks.detach()).sum().item()
        train_all += masks.detach().sum().item()
        # train_mid_acc += (train_pred[:, train_pred.shape[1] // 2].detach() == labels[:, labels.shape[1] // 2]).sum().item()
        train_loss += loss.item()
    
    scheduler.step() # Update the learning rate scheduler
    
    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels, masks = batch
                features = features.to(device)
                labels = labels.to(device)
                masks = masks.to(device)
                # outputs = model(features)
                _, outputs = model(features, masks)
                outputs = [ torch.Tensor(x) for x in outputs ]
                outputs = pad_sequence(outputs, batch_first=True)
                
                # loss = criterion(outputs, labels) 
                # loss = criterion(outputs.transpose(1,-1), labels)
                # _, val_pred = torch.max(outputs, 1) 
                # loss = model.neg_log_likelihood(features, labels)
                loss = model.loss(features, labels, masks)
                # _, val_pred = torch.max(outputs, -1) # get the index of the class with the highest probability
                val_pred = torch.Tensor(outputs).to(device)
                val_acc += ((val_pred.cpu() == labels.cpu()) * masks.cpu()).sum().item()
                val_all += masks.cpu().sum().item()
                # val_mid_acc += (val_pred[:, val_pred.shape[1] // 2].detach() == labels[:, labels.shape[1] // 2]).sum().item()
                val_loss += loss.item()

            # print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
            #     epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            # ))
            
            # print('[{:03d}/{:03d}] Train Acc: {:3.6f} Mid Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} Mid Acc: {:3.6f} Loss: {:3.6f}'.format(
            #     epoch + 1, num_epoch, train_acc/(len(train_set) * concat_nframes), train_mid_acc/len(train_set), train_loss/len(train_loader), 
            #     val_acc/(len(val_set) * concat_nframes), val_mid_acc/len(val_set), val_loss/len(val_loader)
            # ))
            
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/train_all, train_loss/len(train_loader), val_acc/val_all, val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc / val_all > best_acc:
                best_acc = val_acc / val_all
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:3.6f}'.format(best_acc))
                early_stop_count = 0
            else:
                early_stop_count += 1
    else:
        # print('[{:03d}/{:03d}] Train Acc: {:3.6f} Mid Acc: {:3.6f} Loss: {:3.6f}'.format(
        #     epoch + 1, num_epoch, train_acc/(len(train_set) * concat_nframes), 
        #     train_mid_acc/len(train_set), train_loss/len(train_loader)
        # ))
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/train_all, train_loss/len(train_loader)
        ))
        
    if early_stop_count >= early_stop:
        print(f'\nModel is not improving, so we halt the training session.\nSo far best_acc={best_acc}.')
        break

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


100%|██████████| 9/9 [00:09<00:00,  1.03s/it]


[001/800] Train Acc: 0.156096 Loss: 2281.310574 | Val Acc: 0.219772 loss: 2009.323947
saving model with acc 0.219772


100%|██████████| 9/9 [00:09<00:00,  1.05s/it]


[002/800] Train Acc: 0.255450 Loss: 1844.033577 | Val Acc: 0.308191 loss: 1682.288818
saving model with acc 0.308191


100%|██████████| 9/9 [00:09<00:00,  1.05s/it]


...

[121/800] Train Acc: 0.867003 Loss: 214.765331 | Val Acc: 0.806176 loss: 369.576992


100%|██████████| 9/9 [00:09<00:00,  1.09s/it]


[122/800] Train Acc: 0.867246 Loss: 215.324225 | Val Acc: 0.806082 loss: 369.606798


100%|██████████| 9/9 [00:09<00:00,  1.10s/it]

[123/800] Train Acc: 0.867624 Loss: 215.585725 | Val Acc: 0.806040 loss: 369.509118

Model is not improving, so we halt the training session.
So far best_acc=0.8064066220659939.


In [12]:
del train_loader, val_loader
gc.collect()

18

## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [13]:
# load data
# test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, for_rnn=True)
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone')

test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=test_pad_collate_fn)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:00, 3132.35it/s]


In [14]:
# load model
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
# model = RNNClassifier(input_dim=input_dim//concat_nframes, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model = BiLSTMCRF(input_dim=input_dim//concat_nframes, hidden_layers=hidden_layers, hidden_dim=hidden_dim, drop_rate=drop_rate, device=device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [15]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features, masks = batch
        features = features.to(device)
        masks = masks.to(device)

        _, outputs = model(features, masks)
        # outputs = [ torch.Tensor(x) for x in outputs ]
        # outputs = pad_sequence(outputs, batch_first=True)

        # _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        # _, test_pred = torch.max(outputs, -1)
        # test_pred = torch.Tensor(outputs).to(device)
        # test_pred = test_pred[:, test_pred.shape[1] // 2]
        test_pred = np.concatenate(outputs)
        
        pred = np.concatenate((pred, test_pred), axis=0).astype(np.int32)


100%|██████████| 23/23 [00:12<00:00,  1.91it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [16]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))